# AzureAIAgent Usage Examples

This notebook demonstrates how to use the `AzureAIAgent` with various tools, including Bing grounding, file search, and code interpreter.

In [1]:
import asyncio
import os

import dotenv
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.agents.azure._azure_ai_agent import AzureAIAgent
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential

# Load environment variables
dotenv.load_dotenv()

True

## Initialize AzureAIAgent

In [ ]:
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING", None)
)

## Example 1: Using Bing Grounding Tool

In [ ]:
from azure.ai.projects import models

conn = await project_client.connections.get(connection_name=os.getenv("BING_CONNECTION_NAME", None))

bing_tool = models.BingGroundingTool(conn.id)
agent_with_bing_grounding = AzureAIAgent(
    name="bing_agent",
    description="An AI assistant with Bing grounding",
    project_client=project_client,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    tools=bing_tool.definitions,
    metadata={"source": "AzureAIAgent"},
)


async def bing_example():
    result = await agent_with_bing_grounding.on_messages(
        messages=[TextMessage(content="What is Microsoft's annual leave policy?", source="user")],
        cancellation_token=CancellationToken(),
        message_limit=5,
    )
    print(result)


await bing_example()

Response(chat_message=TextMessage(source='bing_agent', models_usage=None, metadata={}, content="Microsoft's annual leave policy, like most companies, can vary based on the employee's role, location, and years of service. However, generally, Microsoft offers a competitive leave policy which typically includes:\n\n1. **Vacation Time**: Microsoft often provides generous vacation days, which can increase with tenure. Employees in the U.S., for example, might start with around 15 vacation days per year and have the potential for more as they remain with the company longer.\n\n2. **Holidays**: Employees are typically given standard public holidays off. This can vary based on the country of employment.\n\n3. **Sick Leave**: Microsoft generally offers paid sick leave to its employees to accommodate health needs.\n\n4. **Personal and Family Leave**: Microsoft also offers additional leave options for personal reasons or to care for family members.\n\n5. **Maternity and Paternity Leave**: Microso

## Example 2: Using File Search Tool

In [4]:
agent_with_file_search = AzureAIAgent(
    name="file_search_agent",
    description="An AI assistant with file search capabilities",
    project_client=project_client,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    tools=["file_search"],
    metadata={"source": "AzureAIAgent"},
)


async def file_search_example():
    await agent_with_file_search.on_upload_for_file_search(
        file_paths=[
            "/workspaces/autogen/python/packages/autogen-core/docs/src/user-guide/core-user-guide/cookbook/data/product_info_1.md"
        ],
        vector_store_name="file_upload_index",
        vector_store_metadata={"source": "AzureAIAgent"},
        cancellation_token=CancellationToken(),
    )
    result = await agent_with_file_search.on_messages(
        messages=[TextMessage(content="Hello, what Contoso products do you know?", source="user")],
        cancellation_token=CancellationToken(),
        message_limit=5,
    )
    print(result)


await file_search_example()

Response(chat_message=TextMessage(source='file_search_agent', models_usage=None, metadata={}, content='I found information about a Contoso product called SmartView Glasses, which belongs to the Contoso Galaxy Innovations brand. This product falls under the category of Smart Eyewear and offers features such as:\n\n- Augmented Reality interface\n- Voice-controlled AI assistant\n- HD video recording with 3D audio\n- UV protection and blue light filtering\n- Wireless charging with extended battery life【4:0†source】.', type='TextMessage'), inner_messages=[])


## Example 3: Using Code Interpreter Tool

In [5]:
agent_with_code_interpreter = AzureAIAgent(
    name="code_interpreter_agent",
    description="An AI assistant with code interpreter capabilities",
    project_client=project_client,
    model="gpt-4o",
    agent_id="asst_LZ0La6Y9LWi2KxAnsrUjRNFw",
    instructions="You are a helpful assistant.",
    tools=["code_interpreter"],
    metadata={"source": "AzureAIAgent"},
)


async def code_interpreter_example():
    await agent_with_code_interpreter.on_upload_for_code_interpreter(
        file_paths="/workspaces/autogen/python/packages/autogen-core/docs/src/user-guide/core-user-guide/cookbook/data/nifty_500_quarterly_results.csv",
        cancellation_token=CancellationToken(),
    )
    result = await agent_with_code_interpreter.on_messages(
        messages=[
            TextMessage(
                content="Aggregate the number of stocks per industry and give me a markdown table as a result?",
                source="user",
            )
        ],
        cancellation_token=CancellationToken(),
    )
    print(result)


await code_interpreter_example()

Response(chat_message=TextMessage(source='code_interpreter_agent', models_usage=None, metadata={}, content='Here is the markdown table showing the number of stocks per industry:\n\n```markdown\n| Industry                            |   Number of Stocks |\n|:------------------------------------|-------------------:|\n| PHARMACEUTICALS                     |                 33 |\n| BANKS                               |                 28 |\n| AUTO PARTS & EQUIPMENT              |                 20 |\n| IT CONSULTING & SOFTWARE            |                 19 |\n| SPECIALTY CHEMICALS                 |                 18 |\n| FINANCE (INCLUDING NBFCS)           |                 17 |\n| ELECTRIC UTILITIES                  |                 14 |\n| INTERNET SOFTWARE & SERVICES        |                 14 |\n| CEMENT & CEMENT PRODUCTS            |                 13 |\n| REALTY                              |                 11 |\n| IRON & STEEL/INTERM.PRODUCTS        |                  9 |\n